**MLSP Project**

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9932774122875479746
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10191896576
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13368355618589838484
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
 xla_global_id: 416903419]

Copy dataset files from the drive, decompress them and combine them into one folder. Each file must have maximum size of 5GB due to the google drive limit. Doing this is because reading each file in the google drive is extremely slow, so must put all the decompressed file locally in the VM to read and process. 


In [ ]:
!cp ./drive/MyDrive/mlsp_project/split_index.zip ./split_index.zip
!unzip -qq "split_index.zip" -d "./"

In [ ]:
!cp ./drive/MyDrive/mlsp_project/dataset/coeffs1.zip ./coeffs1.zip
!cp ./drive/MyDrive/mlsp_project/dataset/coeffs2.zip ./coeffs2.zip
!cp ./drive/MyDrive/mlsp_project/dataset/coeffs3.zip ./coeffs3.zip
!cp ./drive/MyDrive/mlsp_project/dataset/coeffs4.zip ./coeffs4.zip
!cp ./drive/MyDrive/mlsp_project/dataset/coeffs5.zip ./coeffs5.zip
!cp ./drive/MyDrive/mlsp_project/dataset/coeffs6.zip ./coeffs6.zip

In [ ]:
!mkdir coeffs
!mkdir temp
!unzip -qq "coeffs1.zip" -d "./temp"
!mv temp/coeffs1/* coeffs/
!unzip -qq "coeffs2.zip" -d "./temp"
!mv temp/coeffs2/* coeffs/
!unzip -qq "coeffs3.zip" -d "./temp"
!mv temp/coeffs3/* coeffs/


!unzip -qq "coeffs4.zip" -d "./coeffs"
!unzip -qq "coeffs5.zip" -d "./coeffs"
!unzip -qq "coeffs6.zip" -d "./coeffs"

Full length test file

In [ ]:
!cp ./drive/MyDrive/mlsp_project/testfiles.zip ./testfiles.zip
!unzip -qq "testfiles.zip" -d "./"

Define loader for the dataset.


In [ ]:
import torch
from torch.utils.data import Dataset
from numpy import genfromtxt

metadata = './drive/MyDrive/mlsp_project/vox1_meta.csv'
data_path = './coeffs/'
split_path = './split_index/'
file_format = 'csv'

class VoxCeleb1_MFCC(Dataset):
    def __init__(self, is_train, data_length):
        self.is_train = is_train
        self.ids = []
        #very high memory usage to speed up load process
        if is_train:
            f = open(split_path+str(data_length)+'_train.txt', "r")
            lines = f.readlines()
            f.close()
        else:
            f = open(split_path+str(data_length)+'_test.txt', "r")
            lines = f.readlines()
            f.close()
        self.dataset_size = len(lines)
        self.dataset = torch.empty((self.dataset_size,1,41,data_length))
        
        counter = 0
        for line in lines:
            strings = line.split(' ')
            id = int(strings[0])-10001
            self.ids.append(id)
            self.dataset[counter, 0, :, :] = torch.from_numpy(genfromtxt(strings[1].replace('\n',''), delimiter=',').transpose())
            counter += 1
        print(self.dataset.size())

    def __len__(self):
        return self.dataset_size

    def __getitem__(self, idx):
        return self.dataset[idx, :, :, :], self.ids[idx]


full length test dataset

In [ ]:
import torch
from torch.utils.data import Dataset
from numpy import genfromtxt

file_format = 'csv'
testfile_split = './drive/MyDrive/mlsp_project/test_index.txt'
testfile_path = './testfiles/'

train_size = 145265
test_size = 8251

class VoxCeleb1_MFCC_full_len_test(Dataset):
    def __init__(self):
        self.ids = []
        self.dataset_size = test_size
        self.dataset = []
        f = open(testfile_split, "r")
        lines = f.readlines()
        f.close()
        counter = 0
        for line in lines:
          counter += 1
          self.ids.append(int(line)-10001)
          self.dataset.append(torch.from_numpy(genfromtxt(testfile_path+str(counter)+'.csv', delimiter=',').transpose()).unsqueeze(0))

    def __len__(self):
        return self.dataset_size

    def __getitem__(self, idx):
        return self.dataset[idx], self.ids[idx]

Define the network Resnet 34 and put it on gpu

In [ ]:
import torch
import torchvision
import torch.nn as nn

net = torchvision.models.resnet34()
net.conv1  = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=7,stride=2,padding=3,bias=False)
net.fc = nn.Linear(512, 1251)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

net.to(device)

from torchsummary import summary

summary(net.to(device), (1, 41, 100))

Load everything into memory to save the disk reading time and set train loader. This dataset will use a lot of memory

In [ ]:
batch_size = 64
trainloaders = []
for i in range(1,11):
  dataset = VoxCeleb1_MFCC(is_train=True, data_length=i*100) #high memory  
  trainloaders.append(torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2))

torch.Size([39835, 1, 41, 100])
torch.Size([49105, 1, 41, 200])
torch.Size([23712, 1, 41, 300])
torch.Size([12533, 1, 41, 400])
torch.Size([7339, 1, 41, 500])
torch.Size([4291, 1, 41, 600])
torch.Size([2717, 1, 41, 700])
torch.Size([1848, 1, 41, 800])
torch.Size([1231, 1, 41, 900])
torch.Size([2654, 1, 41, 1000])


Full length test

In [ ]:
test_batch_size = 1
testset = VoxCeleb1_MFCC_full_len_test() 
testloader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=True)

100 to 1000 length test

In [ ]:
test_batch_size = 64
testloaders = []
for i in range(1,11):
  dataset = VoxCeleb1_MFCC(is_train=False, data_length=i*100) #high memory  
  testloaders.append(torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2))

torch.Size([2496, 1, 41, 100])
torch.Size([2818, 1, 41, 200])
torch.Size([1357, 1, 41, 300])
torch.Size([612, 1, 41, 400])
torch.Size([364, 1, 41, 500])
torch.Size([225, 1, 41, 600])
torch.Size([137, 1, 41, 700])
torch.Size([75, 1, 41, 800])
torch.Size([50, 1, 41, 900])
torch.Size([117, 1, 41, 1000])


Train

In [ ]:
print(torch.cuda.get_device_properties(0).total_memory/(1024**3))
print(torch.cuda.memory_reserved(0)/(1024**3))
print(torch.cuda.memory_allocated(0)/(1024**3))
import gc
del inputs, labels
gc.collect()
print(torch.cuda.get_device_properties(0).total_memory/(1024**3))
print(torch.cuda.memory_reserved(0)/(1024**3))
print(torch.cuda.memory_allocated(0)/(1024**3))

In [ ]:
import torch.nn as nn
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.97)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
state_dict = torch.load('drive/MyDrive/mlsp_project/Resnet34_test.pth') 
net.load_state_dict(state_dict)
state_dict1 = torch.load('drive/MyDrive/mlsp_project/Resnet34Optimizer_adam.pth')
optimizer.load_state_dict(state_dict1)
state_dict2 = torch.load('drive/MyDrive/mlsp_project/Resnet34Scheduler_adam.pth')
#print(state_dict2)
scheduler.load_state_dict(state_dict2)

In [ ]:
count = 0
if count == 0 or count >= 5:
      count += 1
      continue
    count += 1
    -39835-4291-2717-1848-1231-2654

In [ ]:
import torch
for epoch in range(1, 51):
  epoch_train_loss = 0
  correct = 0

  train_batch_num = 0
  net.train()
  
  for trainloader in trainloaders:
    train_batch_num += len(trainloader)
    #print(len(trainloader))
    for batch_idx, (inputs, labels) in enumerate(trainloader, 0):
      inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device)
      optimizer.zero_grad()
      outputs = net(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      epoch_train_loss += loss.item()
      correct += (outputs.argmax(dim=1) == labels).int().sum()
  scheduler.step()

  net.eval()
  epoch_test_loss = 0
  test_correct = 0
  test_batch_num = len(testloader)
  with torch.no_grad():
    for batch_idx, (inputs, labels) in enumerate(testloader, 0):
      inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device)
      outputs = net(inputs)
      loss = criterion(outputs, labels)

      epoch_test_loss += loss.item()
      test_correct += (outputs.argmax(dim=1) == labels).int().sum()
  print('Test: Loss: {:.7f}\tAccuracy: {}/{}\t \n'.format(\
      epoch_test_loss/test_batch_num, test_correct, test_size))
          
  if epoch % 10 == 0:
    torch.save(net.state_dict(), 'drive/MyDrive/mlsp_project/backup/Resnet34_file{}.pth'.format(str(epoch/10)))
    torch.save(optimizer.state_dict(), 'drive/MyDrive/mlsp_project/backup/Resnet34Optimizer_file{}.pth'.format(str(epoch/10)))
    torch.save(scheduler.state_dict(), 'drive/MyDrive/mlsp_project/backup/Resnet34Scheduler_file{}.pth'.format(str(epoch/10)))
  
  print('Epoch: {}\t Train: Loss: {:.7f}\tAccuracy: {}/{}\t'.format(epoch, epoch_train_loss/train_batch_num, correct, 145265))


Test: Loss: 2.1817950	Accuracy: 4317/8251	 

Epoch: 1	 Train: Loss: 0.2673276	Accuracy: 138421/145265	
Test: Loss: 2.2127919	Accuracy: 4250/8251	 

Epoch: 2	 Train: Loss: 0.2595301	Accuracy: 138882/145265	
Test: Loss: 2.1808694	Accuracy: 4357/8251	 

Epoch: 3	 Train: Loss: 0.2515968	Accuracy: 139230/145265	
Test: Loss: 2.1791603	Accuracy: 4308/8251	 

Epoch: 4	 Train: Loss: 0.2420670	Accuracy: 139650/145265	
Test: Loss: 2.1697702	Accuracy: 4403/8251	 

Epoch: 5	 Train: Loss: 0.2368831	Accuracy: 139862/145265	
Test: Loss: 2.1747328	Accuracy: 4351/8251	 

Epoch: 6	 Train: Loss: 0.2299735	Accuracy: 140247/145265	
Test: Loss: 2.1610565	Accuracy: 4359/8251	 

Epoch: 7	 Train: Loss: 0.2246272	Accuracy: 140478/145265	
Test: Loss: 2.1781749	Accuracy: 4330/8251	 

Epoch: 8	 Train: Loss: 0.2161590	Accuracy: 140836/145265	
Test: Loss: 2.1874877	Accuracy: 4314/8251	 

Epoch: 9	 Train: Loss: 0.2130355	Accuracy: 141069/145265	
Test: Loss: 2.1669827	Accuracy: 4336/8251	 

Epoch: 10	 Train: Loss: 0.20

KeyboardInterrupt: ignored

In [ ]:
torch.save(net.state_dict(), 'drive/MyDrive/mlsp_project/Resnet34_test.pth') #change path if need
torch.save(optimizer.state_dict(), 'drive/MyDrive/mlsp_project/Resnet34Optimizer_adam.pth')
torch.save(scheduler.state_dict(), 'drive/MyDrive/mlsp_project/Resnet34Scheduler_adam.pth')

100 to 1000 Test

In [ ]:
net.eval()
epoch_test_loss = 0
test_correct = 0
test_batch_num = 0
for testloader in testloaders:
  test_batch_num += len(testloader)
  for batch_idx, (inputs, labels) in enumerate(testloader, 0):
    inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device)
    outputs = net(inputs)
    loss = criterion(outputs, labels)

    epoch_test_loss += loss.item()
    test_correct += (outputs.argmax(dim=1) == labels).int().sum()
  print(test_correct)
print('Test: Loss: {:.7f}\tAccuracy: {}/{}\t \n'.format(\
    epoch_test_loss/test_batch_num, test_correct, 8251))

tensor(760, device='cuda:0')
tensor(2071, device='cuda:0')
tensor(2874, device='cuda:0')
tensor(3260, device='cuda:0')
tensor(3497, device='cuda:0')
tensor(3663, device='cuda:0')
tensor(3764, device='cuda:0')
tensor(3823, device='cuda:0')
tensor(3861, device='cuda:0')
tensor(3962, device='cuda:0')
Test: Loss: 2.4303483	Accuracy: 3962/8251	 



Full size test

In [ ]:
state_dict = torch.load('drive/MyDrive/mlsp_project/Resnet34_mfcc.pth') 
net.load_state_dict(state_dict)

In [ ]:
test_batch_size = 1
testset = VoxCeleb1_MFCC_full_len_test() 
testloader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=True)

In [ ]:
net.eval()
epoch_test_loss = 0
test_correct = 0
test_batch_num = len(testloader)
with torch.no_grad():
  for batch_idx, (inputs, labels) in enumerate(testloader, 0):
    inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device)
    outputs = net(inputs)
    loss = criterion(outputs, labels)

    epoch_test_loss += loss.item()
    test_correct += (outputs.argmax(dim=1) == labels).int().sum()
print('Test: Loss: {:.7f}\tAccuracy: {}/{}\t \n'.format(\
    epoch_test_loss/test_batch_num, test_correct, test_size))

Test: Loss: 2.1271773	Accuracy: 4411/8251	 

